# Spatial Weights

Using spatial weights to explore spatial disparities

In [ ]:
import geopandas
import libpysal
import numpy
%matplotlib inline

In [ ]:
df = geopandas.read_file('data/scag_region.gpkg')
df.fillna(df.mean(), inplace=True)

https://www.azavea.com/blog/2019/05/02/analyzing-life-expectancy-in-python/

In [ ]:
wq = libpysal.weights.Queen.from_dataframe(df)

In [ ]:
hv = df['median_home_value'].values

In [ ]:
hv

## Find maximum pairwise difference in hv

In [ ]:
pw_diff = numpy.abs(hv[:, numpy.newaxis] - hv)


In [ ]:
wq.neighbors[0]

In [ ]:
W = wq.full()[0]


In [ ]:
pw_sdiff = W * pw_diff

In [ ]:
pw_sdiff[0, wq.neighbors[0]]

In [ ]:
pw_sdiff.max()

In [ ]:
d_idx = numpy.where(pw_sdiff == pw_sdiff.max())[0]

In [ ]:
hv[d_idx]

In [ ]:
d_idx

In [ ]:
df.iloc[d_idx].plot()

In [ ]:
df.iloc[d_idx].head()

In [ ]:
df.iloc[d_idx]['median_home_value']

In [ ]:
import numpy as np
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [ ]:
 xs = np.sin(np.arange(9)).reshape((3, 3))

In [ ]:
xs

In [ ]:
largest_indices(xs, 3)

In [ ]:
largest_indices(pw_sdiff, 10)

In [ ]:
pw_sdiff[largest_indices(pw_sdiff, 10)]

In [ ]:
idxs = largest_indices(pw_sdiff, 10)

In [ ]:
c_idxs = np.unique(np.concatenate(idxs))

In [ ]:
c_idxs

In [ ]:
df.iloc[c_idxs].plot()

In [ ]:
n,k = df.shape
pair_diff = np.zeros((n,1))
pair_diff[c_idxs] = 1

In [ ]:
df['pair_diff'] = pair_diff

In [ ]:
df.plot(column='pair_diff', categorical='True', legend=True)

In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
m = folium.Map(tiles='cartodbpositron', min_zoom=4, zoom_start=4.25, 
               max_bounds=True,location=[33.8283459,-98.5794797],
               min_lat=5.499550, min_lon=-160.276413, 
               max_lat=83.162102, max_lon=-52.233040)


In [ ]:
df.to_file('data/pairs.geojson',  driver='GeoJSON')

In [ ]:
min_lon, min_lat, max_lon, max_lat = df.total_bounds
c_lon = (max_lon + min_lon)/2
c_lat = (max_lat + min_lat)/2

In [ ]:
m = folium.Map(tiles='cartodbpositron', min_zoom=4, zoom_start=7, 
               max_bounds=True,location=[c_lon, c_lat],
               min_lat=min_lat, min_lon=min_lon, 
               max_lat=max_lat, max_lon=max_lon)
marker_cluster = MarkerCluster(
    options = {'maxClusterRadius':15, 
               'disableCusteringAtZoom':5, 
               'singleMarkerMode':True}).add_to(m)
folium.Choropleth(
    geo_data = 'data/pairs.geojson',
    data = df,
    columns = ['geoid','median_home_value'],
    fill_color = 'YlGn',
    key_on = 'feature.properties.geoid',
    name = 'geojson',
    legend_name='median_home_value'
).add_to(m)
for i, tract in df.iloc[c_idxs].iterrows():   #top_50_tracts_gdf.iterrows():
    x = tract.geometry.centroid.x
    y = tract.geometry.centroid.y
    l = tract.median_home_value
    folium.CircleMarker([y, x], radius=8, color='black', 
                        fill_color='white', fill_opacity=0.5, 
                        tooltip='Median House Value: {}'.format(str(l))).add_to(marker_cluster)
f = folium.Figure()
title = '<h2>Spatial Housing Markets</h2>'
subtitle = '<h4><i>Census tract neighbors across ' + \
           'Southern California with the widest disparities ' + \
           'in home values<i></h4>'
f.html.add_child(folium.Element(title))
f.html.add_child(folium.Element(subtitle))
f.add_child(m)
f

In [ ]:
df.head()